In [ ]:
%cd ..
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import tensorflow as tf

tf.get_logger().setLevel("ERROR")
import tensorflow_datasets as tfds

import tensorflow_datasets_bw as datasets
from tensorflow_datasets_bw import visualize
import dppp

# Load an example dataset and example kernels

In [ ]:
images = (
    tfds.load(name="set14", split="test")
    .map(datasets.get_value("hr"))
    .map(datasets.to_float32)
    .map(datasets.from_255_to_1_range)
)

In [ ]:
tf.random.set_seed(99)  # Random seed for reproducability
noise_stddev = 0.04
image = datasets.get_one_example(images, index=0)[None, ...]
kernel = dppp.conv2D_filter_rgb(dppp.NB_DEBLURRING_LEVIN_KERNELS[0])

# Degrade the image (with wrapping border)

In [ ]:
degraded = dppp.blur(image, kernel, noise_stddev, clip_final=False, mode='wrap')

# Reconstruct the image

In [ ]:
# Load the denoiser
denoiser, (denoiser_min, denoiser_max) = dppp.load_denoiser("models/drugan+_0.0-0.2.h5")

# Callbacks
log_dir = os.path.join("logs", "ppp_nb")
callbacks = [
    # Print the PSNR every 2nd step
    dppp.callback_print_psnr("psnr", 2, image),
    # Log the SSIM to TensorBoard
    dppp.callback_tb_ssim(log_dir, "ssim", image),
]

# Reconstruct the image
reconstructed = dppp.hqs_deblur_nb(
    degraded=degraded,
    kernel=kernel,
    noise_stddev=noise_stddev,
    denoiser=denoiser,
    max_denoiser_stddev=denoiser_max,
    callbacks=callbacks,
)


In [ ]:
# Print PSNR and LPIPS
psnr = dppp.psnr(image, reconstructed).numpy()[0]
lpips = dppp.lpips_alex(image, reconstructed).numpy()[0]
print(f"Reconstructed PSNR: {psnr:0.2f}, LPIPS: {lpips:0.4f}")

# Visualize
visualize.draw_images([image[0], degraded[0], reconstructed[0]])
